<h1 style="color:black" align="center">Линейная классификация</h1>

Мы помним что функционал ошибки выглядел следующим образом:

$\large Q(a, X) = \frac{1}{\ell} \sum\limits_{i=1}^{\ell} [a(x_i) = y_i]$ - долей правильных ответов (accuracy)

Нам будет удобнее решать задачу минимизации, поэтому будем вместо этого использовать долю неправильных ответов:

$\large Q(a, X) = \frac{1}{\ell} \sum\limits_{i=1}^{\ell} [a(x_i) \ne y_i]$ - доля ошибок.

Подставили
Так как этот функционал не дифференцируемый, то его необходимо заменить на функцию, которая оценивает сверху идикатор ошибки. Записали оценку сверху мы следующим образом:

$\large 0 \le \frac{1}{\ell} \sum\limits_{i=1}^{\ell} [y_i \langle w, x \rangle < 0] 
\le 
 \frac{1}{\ell} \sum\limits_{i=1}^{\ell} \tilde L (y_i \langle w, x \rangle) \to \underset{w}{\text{min}}$

<h1 style="color:#008B8B">1. Логистическая регрессия</h1>

<h2 style="color:#008B8B">1.1 Оценивание вероятностей</h2>

### Для чего необходимо оценивать вероятность?

Предположим, имеется модель $\text{b(x)} \in [0, 1]$. Представим, что модель выдала на некотором объекте уверенность равную $0.8$ в классе +1. Как это можно интерпретировать?

Предположим, имеется множество из 10 объектов, где модель возвращает приблизительно одну веренность в своём прогнозе, тогда мы ожидаем, что среди этих объектов 80% будут положительными.

Предположим, мы построили модель, кторая оценивает вероятность положительного класса. Записывать это будем следующим образом:

$\large b(x) \approx p(y = +1 | x)$ - модель оценивает того, что класс положительный для данного объекта $x$. Оценивание веротяности принадлежности объекта к некоторому классу позволяет посмотреть уверенность модели в своём прогнозе. Также бывают задачи, когда нет понимания того, какой ответ в задачи: 

### Пример 1

Пример, мы решаем задачу предсказания кликов пользователя по рекламе, где необходимо предсказать кликнет ли пользователь по данному рекламному банеру. Имеются следующие 3 категориальных признака по которым мы предсказываем нажмёт ли пользователь на рекламный банер.

Так как объект описывается тремя идентификаторами, то может быть ситуация, когда пользователь отправлят один и тот же запрос на который показывается один и тот же банер:

|id пользователя | id запроса | id банера | Нажал ли пользователь? |
|:--------------:|:----------:|:---------:|:----------------------:|
|10              |50          |527        |0                       |
|10              |50          |527        |0                       |
|10              |50          |527        |0                       |
|10              |50          |527        |0                       |
|10              |50          |527        |1                       |

Пользователь может в большинстве случаев не нажимать на рекламу, спустся определённое количество запросов он кликнул на данный банер. Проблема заключается в том что один объект встречается много раз с рзными целевыми переменным (большенство раз 0). Если данную обучающую выборку передать классификатору, который возвращает только класс объекта, тогда ему будет тяжело извлечь закономерность из этой выборки, так как объект один, а ответы целевой переменной разыне. Тогда, можно сделать так, чтобы классификатор возвращал не класс, а вероятность пренадлежность объекта к некоторому классу.

### Пример 2

Представим, что пользователь вводит некоторый запрос, на который необходим показать один из 4 банера. За каждый кликнутый банер мы получаем определённую сумму денег, обазначим как $c(x)$. Показывать самый дорогой банер будет плохой идей, так как пользователь может на него не нажать, для этого необходимо ввести вероятность клика $p(y = +1| x)$ которую оценивает модель.

Что мы максимизируем в данной задаче? Необходимо максимизировать математическое ожидание прибыли, а что такое мат.ожидание прибыли? Это $с(x) \cdot p(y = +1| x)$

|$$c(x)$$|$$p(y = +1| x)$$ |$$с(x) \cdot p(y = +1| x)$$ |
|:------:|:---------------:|:--------------------------:|
|1       |0.5              |0.5                         |
|5       |0.1              |0.5                         |
|10      |0.3              |3                           |
|100     |0.01             |1                           |

Лучшее математическое ожидание будет $3$. Значит данный банер лучше всего показывать. Соответсвенно, данная вероятность помогает выбрать то, что необходимо показывтаь пользователю.

### Пример 3

Банк принимает решение того, какой продукт необходимо предложить пользователю. Имеется продукт мобильный банк, который он подключит с вероятностью 0.9, либо мы можем предложить пользоватлею кредит, на который он согласится с вероятностью 0.1. Если предлогать пользователю продук по вероятности, тогда мы не сможем узнать цену рекомендации. Если на основе вероятности оценить то, сколько мы заработаем, тогда предложить крдеит пользвателю будет более хорошей идеей, так как прибыль намного будет больше.